# Training Notebook

Download the dataset and extract its contents into the root folder, then run the notebook to train the AI and try it.
If you don't want to train the AI, you can extract the pretrained model "svm_model.zip" and run the last cell directly.

In [46]:
import numpy as np
import cv2
import numpy as np
import os
import mediapipe as mp
import time
from tqdm import tqdm

# Train a model

## Load the data

In [48]:
# Define the used directories (same as labels)
label_names = ["Masked", "Unmasked", "Half_Masked"]

# Use it if you want to limit the number of images per category
# to avoid long loading times
# Set it to 0 or less to have no limit
limit = 0

faces = []
labels = []

for id, d in enumerate(label_names):
    print("Begin loading images from " + d)
    time.sleep(.5)
    files = os.listdir('Output/faces/' + d)

    max_files = min(limit, len(files)) if limit > 0 else len(files)
    # Load every image (in the defined limit),
    # resize them to 250x250px
    # and convert them to grayscales
    with tqdm(total=max_files) as pbar:
        for x,f in enumerate(files):
            if x >= max_files:
                break
            faces += [cv2.cvtColor(cv2.resize(cv2.imread('./Output/faces/'+ d + '/' + f), (250, 250), interpolation=cv2.INTER_AREA), cv2.COLOR_BGR2GRAY)]    
            pbar.update(1)

    # Fill the labels according to the index of the directory name
    # Example : 0 for 'Masked', 1 for 'Unmasked' and 2 for 'Half_Masked'
    labels += [id] * (len(faces)-len(labels))

    print("Finished loading images from " + d)

Begin loading images from Masked


100%|██████████████████████████████████████████████████████████████████████████████| 283/283 [00:00<00:00, 1209.41it/s]


Finished loading images from Masked
Begin loading images from Unmasked


100%|██████████████████████████████████████████████████████████████████████████████| 350/350 [00:00<00:00, 1199.90it/s]


Finished loading images from Unmasked
Begin loading images from Half_Masked


100%|██████████████████████████████████████████████████████████████████████████████| 349/349 [00:00<00:00, 1185.13it/s]

Finished loading images from Half_Masked


## Reshape the data

In [49]:
with_mask = []
without_mask = []
half_masked = []

with tqdm(total=len(faces)) as pbar:
    for f, l in zip(faces, labels):
        pbar.update(1)
        
        # transform the image to a numpy array
        data = np.array(f)
        # reshape the image to a linear array
        data = data.reshape(1, -1)
        if l == 0:
            with_mask.append(data)
        elif l == 1:
            without_mask.append(data)
        elif l == 2:
            half_masked.append(data)
        else:
            dumbass.append(data)

with_mask = np.array(with_mask).reshape(len(with_mask), -1)
without_mask = np.array(without_mask).reshape(len(without_mask), -1)
half_masked = np.array(half_masked).reshape(len(half_masked), -1)

print(with_mask.shape)
print(without_mask.shape)
print(half_masked.shape)

# concatenate the arrays 
X = np.r_[with_mask, without_mask, half_masked]
print(X.shape)

labels = np.array(labels)
print(labels.shape)

100%|█████████████████████████████████████████████████████████████████████████████| 982/982 [00:00<00:00, 25758.16it/s]

(283, 62500)
(350, 62500)
(349, 62500)
(982, 62500)
(982,)


## Create the model

In [50]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import joblib

### Split the data for train and test

In [51]:
x_train, x_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=np.random.randint(0,9999))
print(x_train.shape)

(785, 62500)


### Initialize the model

In [52]:
filename = 'svm_model.sav'

# You can load the model if you want to train over the previous trainings
#svm = joblib.load(filename)
svm = SVC(kernel='rbf', C=4, verbose=True)

### Train it

In [53]:
svm.fit(x_train, y_train)

y_pred = svm.predict(x_test)
print(accuracy_score(y_test, y_pred))

[LibSVM]0.8934010152284264


### Save the model

In [54]:
joblib.dump(svm, filename)

['svm_model.sav']

# Try it !

In [55]:
mp_face_detection = mp.solutions.face_detection

svm = joblib.load(filename)

colors = [
    (0, 255, 0),
    (0, 0, 255),
    (0, 255, 255)
]

vid = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(
    model_selection=0, min_detection_confidence=0.6) as face_detection:
    while(vid.isOpened()):#True):
        
        i = 0
        
        # Capture the video frame
        # by frame
        ret, img = vid.read()
        cv2.flip(img, 1, img)
        
        img.flags.writeable = False
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        results = face_detection.process(img)

        # Draw the face detection annotations on the image.
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        if i == 0:
            try:
                if results.detections:
                    for detection in results.detections:
                        # Get the bounding box of the face
                        # and its characteristics
                        bb = detection.location_data.relative_bounding_box
                        x = bb.xmin
                        y = bb.ymin
                        h = bb.height
                        w = bb.width

                        camWidth = img.shape[1]
                        camHeight = img.shape[0]

                        # Convert x, y, h and w from percentages to pixels
                        x = int(x * camWidth)
                        y = int(y * camHeight)
                        h = int(h * camHeight)
                        w = int(w * camWidth)

                        # Resize the face capture and grayscale
                        toPredict = cv2.resize(img[y:y+h, x:x+w], (250, 250), interpolation = cv2.INTER_AREA)
                        toPredict = cv2.cvtColor(toPredict, cv2.COLOR_BGR2GRAY)

                        # Reshape the data
                        toPredict = toPredict.reshape(1, -1)

                        # Predict
                        pred = svm.predict(toPredict)
                        pred = int(pred[0])

                        # if the predicted label is in the list of known labels, draw a rectangle around the face and label it
                        if pred < len(label_names):
                            cv2.rectangle(img,(x,y),(x+w,y+h),colors[pred],2)
                            cv2.putText(img, label_names[pred], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, colors[pred], 3)
            except:
                pass
        i = (i + 1) % 3
        img = cv2.resize(img, (0, 0), fx = 1.5, fy = 1.5)
        
        # Display the resulting frame
        cv2.imshow('frame', img)
        
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()